In [ ]:
%%javascript

IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
import ipywidgets as ipw
from IPython.display import clear_output
from aiida.cmdline.utils.ascii_vis import format_call_graph
import urllib.parse as urlparse
from aiidalab_widgets_base import ProcessFollowerWidget, ProgressBarWidget, viewer

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
pk = urlparse.parse_qs(url.query)['id'][0]
process = load_node(pk)

## Process inputs.

In [ ]:
in_output = ipw.Output()
inp_info = ipw.HTML()
def view_input(change=None):
    with in_output:
        inp_info.value = ''
        clear_output()
        if inpts.value:
            to_dislpay = process.inputs[inpts.value]
            inp_info.value = "PK: {}".format(to_dislpay.id)
            display(viewer(to_dislpay))
        
        
inpts = ipw.Dropdown(
    options=[('Select input', '')] + [(l.title(), l) for l in process.inputs],
    description='Select input:',
    style={'description_width': 'initial'},
    disabled=False,
)
inpts.observe(view_input, names='value')
display(ipw.HBox([inpts, inp_info]), in_output)

## Process outputs.

In [ ]:
out_output = ipw.Output()
out_info = ipw.HTML()
def view_output(change=None):
    with out_output:
        clear_output()
        out_info.value = ''
        if outpts.value:
            to_dislpay = process.outputs[outpts.value]
            out_info.value = "PK: {}".format(to_dislpay.id)
            display(viewer(to_dislpay))
        
outpts = ipw.Dropdown(
    options= [('Select output', '')] + [ (l.title(), l) for l in process.outputs],
#    value='Select output',
    description='Select outputs:',
    style={'description_width': 'initial'},
    disabled=False,
)
outpts.observe(view_output, names='value')
display(ipw.HBox([outpts, out_info]), out_output)

In [ ]:
def calc_info(node):
    """Return a string with the summary of the state of a CalculationNode."""
    from aiida.orm import ProcessNode, WorkChainNode

    if not isinstance(node, ProcessNode):
        raise TypeError('Unknown type: {}'.format(type(node)))

    process_label = node.process_label
    process_state = node.process_state.value.capitalize()
    exit_status = node.exit_status
    path_to_root = '../'
    pk = """<a#space#href={0}aiidalab-widgets-base/process.ipynb?id={1}#space#target="_blank">{1}</a>""".format(path_to_root, node.pk)

    if exit_status is not None:
        string = '{}<{}> {} [{}]'.format(process_label, pk, process_state, exit_status)
    else:
        string = '{}<{}> {}'.format(process_label, pk, process_state)

    if isinstance(node, WorkChainNode) and node.stepper_state_info:
        string += ' [{}]'.format(node.stepper_state_info)

    return string

In [ ]:
follower = ProcessFollowerWidget(process, followers=[ProgressBarWidget])
follower.follow(detach=True)
display(follower)

## Call graph.

In [ ]:
string = format_call_graph(process, info_fn=calc_info)
new_string = string.replace('\n', '<br/>').replace(' ', '&nbsp;').replace('#space#', ' ')
ipw.HTML(new_string)